## Loading Libraries

In [35]:
import pandas as pd; pd.set_option('display.max_columns',100)
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from tqdm.notebook import tqdm

import re

from functools import partial
from scipy.stats import mode

from matplotlib import pyplot as plt; plt.style.use('ggplot')
import seaborn as sns  
import plotly.express as px

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer, PowerTransformer, PolynomialFeatures
from sklearn.pipeline import make_pipeline,Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, RepeatedStratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibrationDisplay, CalibratedClassifierCV
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from collections import Counter
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, HistGradientBoostingClassifier,GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier

import optuna

In [9]:
original=pd.read_csv("Churn_Modelling.csv")
samaple_sub=pd.read_csv("sample_submission.csv")
test=pd.read_csv("test.csv")
train=pd.read_csv("train.csv")

In [12]:
train.head(8).T

,0,1,2,3,4,5,6,7
id,0,1,2,3,4,5,6,7
CustomerId,15674932,15749177,15694510,15741417,15766172,15771669,15692819,15669611
Surname,Okwudilichukwu,Okwudiliolisa,Hsueh,Kao,Chiemenam,Genovese,Ch'ang,Chukwuebuka
CreditScore,668,627,678,581,716,588,593,678
Geography,France,France,France,France,Spain,Germany,France,Spain
Gender,Male,Male,Male,Male,Male,Male,Female,Male
Age,33.0,33.0,40.0,34.0,33.0,36.0,30.0,37.0
Tenure,3,1,10,2,5,4,8,1
Balance,0.0,0.0,0.0,148882.54,0.0,131778.58,144772.69,138476.41
NumOfProducts,2,2,2,1,2,1,1,1


In [13]:
original.head(8).T

,0,1,2,3,4,5,6,7
RowNumber,1,2,3,4,5,6,7,8
CustomerId,15634602,15647311,15619304,15701354,15737888,15574012,15592531,15656148
Surname,Hargrave,Hill,Onio,Boni,Mitchell,Chu,Bartlett,Obinna
CreditScore,619,608,502,699,850,645,822,376
Geography,France,Spain,France,France,Spain,Spain,NaN,Germany
Gender,Female,Female,Female,Female,Female,Male,Male,Female
Age,42.0,41.0,42.0,39.0,43.0,44.0,50.0,29.0
Tenure,2,1,8,1,2,8,7,4
Balance,0.0,83807.86,159660.8,0.0,125510.82,113755.78,0.0,115046.74
NumOfProducts,1,1,3,2,1,2,2,4


In [15]:
train.shape,original.shape,test.shape

((165034, 14), (10002, 14), (110023, 13))

In [19]:
original.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          1
Gender             0
Age                1
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          1
IsActiveMember     1
EstimatedSalary    0
Exited             0
dtype: int64

In [22]:
train.nunique()

id                 165034
CustomerId          23221
Surname              2797
CreditScore           457
Geography               3
Gender                  2
Age                    71
Tenure                 11
Balance             30075
NumOfProducts           4
HasCrCard               2
IsActiveMember          2
EstimatedSalary     55298
Exited                  2
dtype: int64